In [1]:
 import tensorflow as tf
import pandas as pd
import numpy as np
import sklearn
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

In [2]:
def plot_graphs(history, string):
    plt.plot(history.history[string])
    # plt.plot(history.history['val_'+string], '')
    plt.xlabel("Epochs")
    plt.ylabel(string)
    # plt.legend([string, 'val_'+string])
    plt.show()

In [3]:
df = pd.read_csv('oversampled_CICIDS2017_Wed.csv')
df.head()

C:\Users\kdagi\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3058: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,...,Active Mean,Active Std,Active Max,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,target
0,486607,192.168.10.3-192.168.10.14-53-53558-17,192.168.10.14,53558,192.168.10.3,53,17,5/7/2017 3:10,48246,2,...,0.0,0.00000,0.0,0.0,0.000000e+00,0.00000,0.0,0.0,BENIGN,0
1,63680,192.168.10.14-23.21.72.218-51518-443-6,192.168.10.14,51518,23.21.72.218,443,6,5/7/2017 10:36,60337508,12,...,73007.0,54735.42036,184735.0,50540.0,9.983217e+06,30901.22027,10000000.0,9923615.0,BENIGN,0
2,290590,172.16.0.1-192.168.10.50-42900-80-6,172.16.0.1,42900,192.168.10.50,80,6,5/7/2017 10:58,100790813,5,...,9998.0,0.00000,9998.0,9998.0,1.010000e+08,0.00000,101000000.0,101000000.0,DoS Hulk,1
3,589281,151.101.208.207-192.168.10.16-443-42038-6,192.168.10.16,42038,151.101.208.207,443,6,5/7/2017 9:31,3,2,...,0.0,0.00000,0.0,0.0,0.000000e+00,0.00000,0.0,0.0,BENIGN,0
4,188930,172.16.0.1-192.168.10.50-47886-80-6,172.16.0.1,47886,192.168.10.50,80,6,5/7/2017 10:50,3,2,...,0.0,0.00000,0.0,0.0,0.000000e+00,0.00000,0.0,0.0,DoS Hulk,1


In [7]:
df2 = df.groupby([' Timestamp'])[' Flow Duration'].count()
df2 = pd.DataFrame(df2).reset_index()
df2.columns=[' Timestamp','BiFlowsCount']
df = df.merge(df2, left_on=' Timestamp', right_on=' Timestamp')
#df = df.sort_values(' Timestamp')

df

,Unnamed: 0,Flow ID,Source IP,Source Port,Destination IP,Destination Port,Protocol,Timestamp,Flow Duration,Total Fwd Packets,...,Active Min,Idle Mean,Idle Std,Idle Max,Idle Min,Label,target,BiFlowsCount_x,BiFlowsCount_y,BiFlowsCount
0,486607,192.168.10.3-192.168.10.14-53-53558-17,192.168.10.14,53558,192.168.10.3,53,17,5/7/2017 3:10,48246,2,...,0.0,0.0,0.000,0.0,0.0,BENIGN,0,3042,3042,3042
1,450780,149.56.241.113-192.168.10.5-443-55081-6,192.168.10.5,55081,149.56.241.113,443,6,5/7/2017 3:10,63318690,42,...,66069.0,6950374.0,1911561.654,10000000.0,5110560.0,BENIGN,0,3042,3042,3042
2,448060,192.168.10.8-192.28.146.84-60483-443-6,192.168.10.8,60483,192.28.146.84,443,6,5/7/2017 3:10,284,3,...,0.0,0.0,0.000,0.0,0.0,BENIGN,0,3042,3042,3042
3,610827,162.208.20.178-192.168.10.5-443-55058-6,192.168.10.5,55058,162.208.20.178,443,6,5/7/2017 3:10,6931,1,...,0.0,0.0,0.000,0.0,0.0,BENIGN,0,3042,3042,3042
4,632511,192.168.10.3-192.168.10.51-53-25630-17,192.168.10.51,25630,192.168.10.3,53,17,5/7/2017 3:10,180,2,...,0.0,0.0,0.000,0.0,0.0,BENIGN,0,3042,3042,3042
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
799995,571845,192.168.10.3-40.70.221.249-56540-443-6,40.70.221.249,443,192.168.10.3,56540,6,5/7/2017 5:08,43,1,...,0.0,0.0,0.000,0.0,0.0,BENIGN,0,29,29,29
799996,664632,192.168.10.14-40.70.221.249-58668-443-6,40.70.221.249,443,192.168.10.14,58668,6,5/7/2017 5:08,10,1,...,0.0,0.0,0.000,0.0,0.0,BENIGN,0,29,29,29
799997,515236,192.168.10.15-209.48.71.129-58122-80-6,209.48.71.129,80,192.168.10.15,58122,6,5/7/2017 5:08,1,2,...,0.0,0.0,0.000,0.0,0.0,BENIGN,0,29,29,29
799998,528475,192.168.10.3-192.168.10.50-53-65345-17,192.168.10.50,65345,192.168.10.3,53,17,5/7/2017 5:08,200,2,...,0.0,0.0,0.000,0.0,0.0,BENIGN,0,29,29,29


In [8]:
df.columns

Index(['Unnamed: 0', 'Flow ID', ' Source IP', ' Source Port',
       ' Destination IP', ' Destination Port', ' Protocol', ' Timestamp',
       ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets',
       'Total Length of Fwd Packets', ' Total Length of Bwd Packets',
       ' Fwd Packet Length Max', ' Fwd Packet Length Min',
       ' Fwd Packet Length Mean', ' Fwd Packet Length Std',
       'Bwd Packet Length Max', ' Bwd Packet Length Min',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s',
       ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max',
       ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std',
       ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean',
       ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags',
       ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags',
       ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s',
       ' Bwd Packets/s', ' Min Packet Len

In [9]:
del df['Unnamed: 0']
del df['target']

In [12]:
 features = ['Flow ID', ' Source IP', ' Source Port',
       ' Destination IP', ' Destination Port', ' Protocol', ' Timestamp',
       ' Flow Duration', ' Total Fwd Packets', ' Total Backward Packets',
       'Total Length of Fwd Packets', ' Total Length of Bwd Packets',
       ' Fwd Packet Length Max', ' Fwd Packet Length Min',
       ' Fwd Packet Length Mean', ' Fwd Packet Length Std',
       'Bwd Packet Length Max', ' Bwd Packet Length Min',
       ' Bwd Packet Length Mean', ' Bwd Packet Length Std', 'Flow Bytes/s',
       ' Flow Packets/s', ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max',
       ' Flow IAT Min', 'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std',
       ' Fwd IAT Max', ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean',
       ' Bwd IAT Std', ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags',
       ' Bwd PSH Flags', ' Fwd URG Flags', ' Bwd URG Flags',
       ' Fwd Header Length', ' Bwd Header Length', 'Fwd Packets/s',
       ' Bwd Packets/s', ' Min Packet Length', ' Max Packet Length',
       ' Packet Length Mean', ' Packet Length Std', ' Packet Length Variance',
       'FIN Flag Count', ' SYN Flag Count', ' RST Flag Count',
       ' PSH Flag Count', ' ACK Flag Count', ' URG Flag Count',
       ' CWE Flag Count', ' ECE Flag Count', ' Down/Up Ratio',
       ' Average Packet Size', ' Avg Fwd Segment Size',
       ' Avg Bwd Segment Size', ' Fwd Header Length.1', 'Fwd Avg Bytes/Bulk',
       ' Fwd Avg Packets/Bulk', ' Fwd Avg Bulk Rate', ' Bwd Avg Bytes/Bulk',
       ' Bwd Avg Packets/Bulk', 'Bwd Avg Bulk Rate', 'Subflow Fwd Packets',
       ' Subflow Fwd Bytes', ' Subflow Bwd Packets', ' Subflow Bwd Bytes',
       'Init_Win_bytes_forward', ' Init_Win_bytes_backward',
       ' act_data_pkt_fwd', ' min_seg_size_forward', 'Active Mean',
       ' Active Std', ' Active Max', ' Active Min', 'Idle Mean', ' Idle Std',
       ' Idle Max', ' Idle Min', 'BiFlowsCount_x',
       'BiFlowsCount_y', 'BiFlowsCount'] 

In [19]:
fraction = 0.5
df = df.sample(frac=fraction, replace=True, random_state=1)

In [20]:
def targetify(s):
    if s == 'BENIGN':
        return 0
    else:
        return 1

In [21]:
df['Target']=df[' Label'].apply(targetify)

In [22]:
X = df[features]
X[features] = X[features].apply(pd.to_numeric, errors='coerce', axis=1)

In [23]:
y = df['Target']

In [24]:
X = X.fillna(0)

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
print (X_train.shape, y_train.shape)
print( X_test.shape, y_test.shape)

(320000, 87) (320000,)
(80000, 87) (80000,)


In [26]:
min_max_scaler = preprocessing.MinMaxScaler()
x_scaled = min_max_scaler.fit_transform(X_train.values)
X_train = pd.DataFrame(x_scaled,columns=features)
x_scaled_test = min_max_scaler.fit_transform(X_test.values)
X_test = pd.DataFrame(x_scaled_test,columns=features)

In [27]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(100000, 64), # since it doesn't consider "words," the embedding doesn't really matter
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    # tf.keras.layers.Dense(1, activation='softmax') # loss too big
]) 

In [28]:
model.compile(loss='binary_crossentropy',
              # optimizer='sgd', # almost same
              optimizer=tf.keras.optimizers.Adam(1e-4),
              metrics=['accuracy'])

In [29]:
history = model.fit(X_train.values, y_train.values, epochs=3)


Train on 320000 samples
Epoch 1/3
320000/320000 [==============================] - 1132s 4ms/sample - loss: 0.4024 - accuracy: 0.8153
Epoch 2/3
320000/320000 [==============================] - 1158s 4ms/sample - loss: 0.3186 - accuracy: 0.8780
Epoch 3/3
320000/320000 [==============================] - 1154s 4ms/sample - loss: 0.3187 - accuracy: 0.8775


In [30]:
pred_class = model.predict(X_test.values[:])
print(pred_class)

[[0.27193978]
 [0.86086416]
 [0.86086416]
 ...
 [0.9910575 ]
 [0.86086434]
 [0.34109417]]


In [31]:
def myRound(x, r):
    if x>r/float(1000):
        return 1
    else:
        return 0

In [32]:
print(round(pred_class[2][0]))
print(round(pred_class[1][0]))

1.0
1.0


In [33]:
predictions = [int(round(x[0])) for x in pred_class]
true_class = list(y_test)

In [34]:
print(true_class[:10], len(true_class))
print(predictions[:10], len(predictions))

[0, 1, 0, 1, 0, 0, 0, 1, 0, 1] 80000
[0, 1, 1, 1, 0, 0, 0, 1, 0, 1] 80000


In [35]:
compdf = pd.DataFrame({'pred_class':predictions, 'true_class':true_class})
compdf = compdf.sort_values('pred_class', ascending=False)
print(str(compdf.head(10)))

       pred_class  true_class
40000           1           0
66174           1           0
28889           1           1
28890           1           1
66173           1           1
47366           1           1
66171           1           1
28894           1           1
47365           1           1
47364           1           1


In [36]:
predictions = list(compdf['pred_class'].apply(myRound, r=225))
true_class = list(compdf['true_class'])

In [37]:
print(predictions[:10])
print(true_class[:10])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0, 0, 1, 1, 1, 1, 1, 1, 1, 1]


In [38]:
from sklearn.metrics import confusion_matrix

In [39]:
confm = confusion_matrix(true_class, predictions)
confm

array([[34421,  5528],
       [ 4004, 36047]], dtype=int64)

In [40]:
tn, fp, fn, tp = confm.ravel()
(tn + tp)/(tn+tp+fn+fp)

0.88085

In [41]:
print("False alarm rate is")
fp/(tn+fp)

False alarm rate is


0.13837642994818394

In [42]:
print("Precision is")
Precision = tp/(tp+fp)
Precision

Precision is


0.8670354780517138

In [43]:
print("Recall is")
Recall = tp/(tp+fn)
Recall

Recall is


0.9000274649821478

In [44]:
print("F1 is")
2*Precision*Recall/(Precision+Recall)

F1 is


0.8832234827138412